In [84]:
from importlib import reload
from functools import reduce
import utils
import warnings

In [189]:
reload(utils)

<module 'utils' from '/Users/nelson/Documents/NYBG/BAD2matrix/utils.py'>

In [87]:
infiles = ['alg_test_0.fasta', 'alg_test_1.fasta', 'alg_test_2.fasta']

dummy = ['''
>sp0#sample0
TATTCCTCTATTA---GTAATTGGGCTTCTACTT-TTCCAAGAGCAACTAAAAATATTCGGCGTATC---
>sp1_0#sample0
-ATTCCTCTATTA---GTAATTGGGCTTCTACTT-TTCCATGAGCAACTAAAAATATACGGCGTATCT--
>sp2#sample0
--TTCCTCTATTAA-AGGAATTGGG--TCTACATTTTCCACGAGCAACTACCCATATTCGGCGTATCTG-
>sp3#sample0
---TCCTCTATTAA-AGGAATTGGG--TCTACATTTTCCAAGAGCAACTACCCATATTC-GCGTATCTGG
''',
'''
>sp0#sample0
CTTCTTTGCATTTATTACGATCGATTCTCCATGAATG------TAGTTTTAGTAAAGAAAATTTGCAGAAATCTCTGATT
>sp1_1#sample1
CTTCGTTGCATTTATTACGATC-ATTCTCCATGAATG------TAGTTTTAGTAAAGAATATTTGCAGAAATCTCTGACT
>sp2#sample0
CTTCGCTGCAT--ATTACGATC-ATTCTCCATGAATG------TAGTTTTAGTTAAGAATATTTGCAGAAATCTCTGACT
>sp4#sample0
CTTCGCTGCAT--ATTACGATC-ATTCTCCATGAATGATAATCTAGTTTTAGTTAAGAATATTTGCAGAAATCTCTGATT
''',
'''
>sp0#sample0
TATTCCTCTATTA---GTAATTGGGCTTCTACTT-TTCCAAGAGGGCGGAAAAATATTCGGCGTATC---
>sp5#sample0
TATTCCTCTATTA---GTAATTGGGCTTCTACTT-TTCCAAGAGCGGGGAAAAATATTCGGCGTATC---
'''
]

for inf, du in zip(infiles, dummy):
	with open(inf, 'w') as fh:
		fh.write(du)

In [205]:
root_name = 'testito'

In [206]:
raxml_main = root_name + '.phy'
raxml_part = root_name + '.part'
(name_map, act_files) = utils.get_name_map(infiles, True, 1)
term_names = sorted(list(set(name_map.values()))) # Why sort should be done in reverse order?
longest = max(term_names, key = len)
spp_data = {name: utils.Term_data(name) for name in term_names}
non_informative_partitions = []
final_spp_count = 0


In [210]:
!rm temporary*
!rm testito.phy

zsh:1: no matches found: temporary*


In [211]:
code_indels = True

for file in act_files:

	partition = utils.Partition(file, name_map)

	if code_indels:
		partition.indel_coder()

	partition.informative_stats()
	tot_inf = len(reduce(lambda x, y: x + y, partition.metadata["informative_chars"]))

	if tot_inf == 0:
		warnings.warn(f"Dataset in file {file} has no informative characters, therefore it will not be further processed and its data completelly excluded from the output files.")
		non_informative_partitions.append(file)

	else:
		# Parse all data to each species file		
		for name in spp_data:
			spp_data[name].feed(partition)

# remove uninformative files and spp 
if not code_indels:
	act_files = [x for x in act_files if not x in non_informative_partitions]
else:
	to_rm = []
	for file in act_files:
		if file in non_informative_partitions and f"{file}_indels" in non_informative_partitions:
			to_rm.append(file)
	act_files = [x for x in act_files if not x in to_rm]

spp_data = {sp: spp_data[sp] for sp in spp_data if 
			len([x for x in spp_data[sp].metadata["presence"] if x]) > 0}

# Write matrices to files

# Write headers
tot_size = sum([x for x in spp_data[next(iter(spp_data))].metadata["size"]])
raxml_header = f" {len(spp_data)} {tot_size} \n"

with open(raxml_main, "w") as oh:
	oh.write(raxml_header)

<ipython-input-211-266f651aae46>:14: UserWarning: Dataset in file alg_test_2.fasta has no informative characters, therefore it will not be further processed and its data completelly excluded from the output files.
  warnings.warn(f"Dataset in file {file} has no informative characters, therefore it will not be further processed and its data completelly excluded from the output files.")


In [213]:
act_files

['alg_test_0.fasta', 'alg_test_1.fasta', 'alg_test_2.fasta']

In [208]:
spp_data

{'sp0_sample0': <utils.Term_data at 0x7f9b408f1d60>,
 'sp1_sample0': <utils.Term_data at 0x7f9b408f1bb0>,
 'sp1_sample1': <utils.Term_data at 0x7f9b107bdd00>,
 'sp2_sample0': <utils.Term_data at 0x7f9b106f1cd0>,
 'sp3_sample0': <utils.Term_data at 0x7f9b106f13d0>,
 'sp4_sample0': <utils.Term_data at 0x7f9b10b99a00>,
 'sp5_sample0': <utils.Term_data at 0x7f9b10b99760>}

In [198]:
print(tot_size)
print(raxml_header)

314
 6 314 



In [203]:
spp_data['sp4_sample0'].metadata

{'size': [70, 4, 80, 3, 70, 4, 80, 3],
 'type': ['nucleic',
  'indel',
  'nucleic',
  'indel',
  'nucleic',
  'indel',
  'nucleic',
  'indel'],
 'informative_chars': [[17, 32, 50, 51, 52],
  [1, 2, 3],
  [5, 53, 78],
  [0],
  [17, 32, 50, 51, 52],
  [1, 2, 3],
  [5, 53, 78],
  [0]],
 'presence': [False, False, True, True, False, False, True, True],
 'origin': ['alg_test_0.fasta',
  'alg_test_0.fasta_indels',
  'alg_test_1.fasta',
  'alg_test_1.fasta_indels',
  'alg_test_0.fasta',
  'alg_test_0.fasta_indels',
  'alg_test_1.fasta',
  'alg_test_1.fasta_indels']}

In [188]:
!rm raxml_test.phy
for sp in spp_data:
		spp_data[sp].parse_raxml("raxml_test.phy")